In [189]:
import pandas as pd
import numpy as np
from datetime import datetime

In [190]:
df = pd.read_csv("../../data/trusted/arquivo_anonimizado_v2.csv")
D_ALUNO = pd.read_csv('../../data/refined/d_aluno.csv')
D_CURSO = pd.read_csv('../../data/refined/d_curso.csv')
D_PERIODO = pd.read_csv('../../data/refined/d_periodo.csv')

In [191]:
# Converte a coluna 'cr_periodo' para string
df["cr_periodo"] = df["cr_periodo"].astype(str)

# Converte a coluna 'cr_periodo' para uma lista de tuplas
df["cr_periodo"] = df["cr_periodo"].apply(
    lambda x: [tuple(item.split(" - ")) for item in x.split("\n")]
)
# Explode a coluna 'cr_periodo' para dividir as listas em linhas separadas
df_exploded = df.explode("cr_periodo")

# Cria um novo DataFrame com as tuplas da coluna 'crPorPeriodo' separadas em colunas 'periodo' e 'cr_periodo'
df_crPeriodo = pd.DataFrame(
    df_exploded["cr_periodo"].tolist(),
    index=df_exploded.index,
    columns=("periodo", "cr_periodo"),
)

# Adiciona as colunas 'matricula_dre' e 'periodo_temp' ao novo DataFrame
df_crPeriodo["matricula_dre"] = df_exploded["matricula_dre"]

df_crPeriodo.dropna(inplace=True)

# CRA

In [192]:
# Converte a coluna 'cra_periodo' para string
df["cra_periodo"] = df["cra_periodo"].astype(str)

# Converte a coluna 'crPorPeriodo' para uma lista de tuplas
df["cra_periodo"] = df["cra_periodo"].apply(
    lambda x: [tuple(item.split(" - ")) for item in x.split("\n")]
)
# Explode a coluna 'crPorPeriodo' para dividir as listas em linhas separadas
df_exploded = df.explode("cra_periodo")

# Cria um novo DataFrame com as tuplas da coluna 'crPorPeriodo' separadas em colunas 'periodo' e 'cr_periodo'
df_craPeriodo = pd.DataFrame(
    df_exploded["cra_periodo"].tolist(),
    index=df_exploded.index,
    columns=("periodo", "cr_acumulado"),
)
# Adiciona as colunas 'matriculaDRE' e 'periodo_temp' ao novo DataFrame
df_craPeriodo["matricula_dre"] = df_exploded["matricula_dre"]
df_craPeriodo.dropna(inplace=True)
df_craPeriodo.drop_duplicates(inplace=True)

# MERGER

In [193]:
df = df.drop(['cr_periodo', 'cra_periodo'], axis=1)

In [194]:
df_crs = df_craPeriodo.merge(df_crPeriodo, how='outer')

In [195]:
f_situacao_periodo = df.merge(df_crs)

In [196]:
f_situacao_periodo =f_situacao_periodo.merge(D_ALUNO, on='matricula_dre')

In [197]:
f_situacao_periodo = f_situacao_periodo.merge(D_PERIODO, on='periodo')

In [198]:
f_situacao_periodo = f_situacao_periodo.merge(D_CURSO)

In [199]:
f_situacao_periodo = f_situacao_periodo[['sk_d_aluno', 'sk_d_periodo', 'sk_d_curso', 'cr_periodo', 'cr_acumulado']]

In [200]:
f_situacao_periodo.to_csv('../../data/refined/f_situacao_periodo.csv', index=False)